In [20]:
import pandas as pd
import numpy as np
import spacy
import string
from bs4 import BeautifulSoup
import re
import geonamescache

In [2]:
df = pd.read_csv("..\data\IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df.shape

(50000, 2)

In [4]:
df.dtypes

review       object
sentiment    object
dtype: object

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


there arn't any null values in the both columns

In [6]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [7]:
df.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

the data is completely balanced

In [8]:
# # removing urls
# def remove_url(text):
#     re.sub(r'http\S+', '', text)

# # redundant function to clean all html tags
# def clean_html(text):
#     clean = re.compile('<,*?>')
#     return re.sub(clean,'',text)


#Define function for removing special characters(punctuation)
# def remove_special_characters(text):
#     pattern=r'[^a-zA-z0-9\s]'
#     text=re.sub(pattern,'',text)
#     return text

#    text = re.sub('\W+',' ', text)


In [9]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()


def remove_punctuation(text):  #removing special characters(punctuation)
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


#Removing the noisy text
def clean_text(text):
    text = strip_html(text)
    text = remove_punctuation(text)
    return text
#Apply function on review column
# df['review'] = [clean_text(text) for text in df['review']]
df['review'] = df['review'].apply(clean_text)

In [16]:
!pip show spacy

Name: spacy
Version: 3.3.1
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: c:\users\huzyfa\anaconda3\lib\site-packages
Requires: blis, catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, pathy, preshed, pydantic, requests, setuptools, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, wasabi
Required-by: en-core-web-lg


In [18]:
nlp = spacy.load('en_core_web_lg')
ner = nlp.get_pipe('ner')
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [67]:
dt = pd.DataFrame(columns=["names", "Entity"])
dt

,names,Entity


In [24]:
spacy.explain("GPE")
spacy.explain ("PERSON")

'People, including fictional'

In [47]:
# gets dictionary for city names
gc = geonamescache.GeonamesCache()      
cities = gc.get_cities()
cities = {v:0 for _, val in cities.items() for k, v in val.items() if k=='name'}

In [73]:
dt = pd.DataFrame()
for text in df.review.loc[:50]:
    doc = nlp(text)
    for ent in doc.ents:
        if (ent.label_=="PERSON"):
            dt.append([ent.text, "Person"])   
        if (ent.label_=="GPE"):
            if ent.text in cities:
                dt.append({ent.text, "City"], ignore_index = True) 
dt

""


In [53]:
dt.to_json("..\data\ner.json", orient="records")

OSError: [Errno 22] Invalid argument: '..\\data\ner.json'